In [9]:
import toughio
import numpy as np
import sys

# --- CONFIGURATION ---
mesh_file = "MESH"
output_gener = "GENER"

# CRITICAL: Update this to your well location in real coordinates (UTM)
# Based on your previous logs, your mesh is in the 286,000 range.
injection_pos = [325608.3, 5097232.8, -100.0] 
injection_rate = 1.0  # kg/s
# ---------------------

print(f"Reading {mesh_file}...")
try:
    mesh_data = toughio.read_mesh(mesh_file)
except Exception as e:
    print(f"Error reading mesh: {e}")
    sys.exit(1)

# --- ROBUST DATA EXTRACTION ---
# Handle: 1. Mesh Object, 2. Dict with 'ELEME', 3. Dict with 'elements'
try:
    if not isinstance(mesh_data, dict):
        # Case A: It's a Mesh Object (Newer toughio)
        centers = mesh_data.centers
        labels = mesh_data.labels
    else:
        # Case B: It's a Dictionary (Older toughio / specific format)
        # Check which key holds the block data
        if 'elements' in mesh_data:
            eleme_dict = mesh_data['elements']
        elif 'ELEME' in mesh_data:
            eleme_dict = mesh_data['ELEME']
        else:
            raise KeyError(f"Could not find element data. Found keys: {list(mesh_data.keys())}")
        
        # Extract data from the found dictionary
        labels = np.array(list(eleme_dict.keys()))
        centers = np.array([v['center'] for v in eleme_dict.values()])

except Exception as e:
    print(f"Error parsing mesh data: {e}")
    sys.exit(1)

# --- FIND CLOSEST BLOCK ---
print(f"Searching for block closest to {injection_pos}...")
dist = np.linalg.norm(centers - injection_pos, axis=1)
idx = np.argmin(dist)
closest_dist = dist[idx]
block_name = labels[idx]

print(f"   -> Found Block: '{block_name}'")
print(f"   -> Distance:    {closest_dist:.2f} m")

if closest_dist > 100.0:
    print("\n[WARNING] Well is >100m from target! Check coordinates.")

# --- WRITE GENER FILE (MANUAL) ---
print(f"Writing {output_gener}...")
with open(output_gener, "w") as f:
    f.write("GENER\n")
    # Fixed Format: Name(5) Block(5) Comp(5) LTAB(5) Rate(10.3E)
    f.write(f"{'INJ01':<5}{block_name:<5}{'COM3':<5}{'':<5}{injection_rate:10.3E}\n")
    f.write("\n")

print("Done.")

Reading MESH...
Searching for block closest to [325608.3, 5097232.8, -100.0]...
   -> Found Block: 'A3M65'
   -> Distance:    706.54 m

[WARNING] Well is >100m from target! Check coordinates.
Writing GENER...
Done.
